In [1]:
import torch
import numpy as np 
import os
import pandas as pd
import pickle
# creating dataloader
from dataloader import MeltomeUnirepDataset 

In [4]:
# tester code
URdir = "../datasets/UniRep_Vecs"
dict_path = "../datasets/protID2MT.p"
MUData = MeltomeUnirepDataset(URdir=URdir, dictPath=dict_path)